<a href="https://colab.research.google.com/github/reino-juan/streamlit-example/blob/master/Destcargar_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import streamlit as st
from pytube import YouTube
from moviepy.video.io.VideoFileClip import VideoFileClip

# Set download path
download_path = "/app"

# Define function to crop video
def crop_video(input_file, output_file, start_time, end_time):
    # Convert start and end times to seconds
    start_seconds = sum(x * int(t) for x, t in zip([3600, 60, 1], start_time.split(":")))
    end_seconds = sum(x * int(t) for x, t in zip([3600, 60, 1], end_time.split(":")))

    # Crop the video using moviepy
    clip = VideoFileClip(input_file).subclip(start_seconds, end_seconds)

    # Export the cropped video
    clip.write_videofile(output_file, codec='libx264')

    # Return output file path
    return output_file

# Define Streamlit app
def main():
    st.title("YouTube Video Cropper")

    # Get YouTube video URL from user input
    youtube_url = st.text_input("Enter YouTube Video URL")

    # Check if URL is valid
    if "youtube.com" in youtube_url:
        try:
            # Download YouTube video
            yt = YouTube(youtube_url)
            yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(download_path)

            # Get video file path and name
            input_file = os.path.join(download_path, yt.title + ".mp4")

            # Get start and end times from user input
            start_time = st.text_input("Enter Start Time (hh:mm:ss)")
            end_time = st.text_input("Enter End Time (hh:mm:ss)")

            # Check if start and end times are valid
            if start_time and end_time:
                try:
                    # Crop video
                    output_file = os.path.join(download_path, yt.title + "_cropped.mp4")
                    output_file = crop_video(input_file, output_file, start_time, end_time)

                    # Download cropped video
                    st.success("Video cropped successfully!")
                    st.download_button("Download Cropped Video", output_file)
                except Exception as e:
                    st.error(f"Error cropping video: {str(e)}")
            else:
                st.warning("Please enter valid start and end times")
        except Exception as e:
            st.error(f"Error downloading YouTube video: {str(e)}")
    else:
        st.warning("Please enter a valid YouTube video URL")

if __name__ == "__main__":
    main()
